In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary

In [3]:
import import_ipynb
from datagen import Data

importing Jupyter notebook from datagen.ipynb


In [19]:

seglength = 9000
train_ratio = 0.8
pnes_train, pnes_test, pnes_train_labels, pnes_test_labels = Data('D:\EEG\PNES', train_ratio, seglength, class_num = 3)
temporal_train, temporal_test, temporal_train_labels, temporal_test_labels = Data('D:\EEG\TLE', train_ratio, seglength, class_num = 4)


In [20]:

val_ratio = 0.1
X_train = torch.cat((pnes_train, temporal_train))
Y_train = torch.cat((pnes_train_labels, temporal_train_labels))

X_test = torch.cat((pnes_test, temporal_test))
Y_test = torch.cat((pnes_test_labels, temporal_test_labels))

X_val = X_train[:int(val_ratio * len(Y_train))]
Y_val = Y_train[:int(val_ratio * len(Y_train))]

X_train = X_train[int(val_ratio * len(Y_train)):]
Y_train = Y_train[int(val_ratio * len(Y_train)):]


In [36]:
np.shape(X_train)

torch.Size([6675, 1, 9000, 20])

In [67]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.T = 9000

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 20), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4496, 4)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        x = x.reshape(-1)
        x = F.sigmoid(self.fc1(x))
        return x



model = CNN()        

model.forward(torch.Tensor(np.random.rand(1, 1, 9000, 20)))
#summary(model.cuda(), (1, 1, 9000, 20))


tensor([0.8650, 0.4775, 0.8916, 0.4809], grad_fn=<SigmoidBackward0>)

In [ ]:
batch_size = 32
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

for epoch in range(10):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = X_train[s:e]
        labels = torch.FloatTensor(Y_train[s:e].T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data[0]

In [38]:
x = torch.randn(4, 100, 120, 20)
y = nn.Conv2d(100, 20, 2)(x)
np.shape(y)

torch.Size([4, 20, 119, 19])